# API Wrapper for USDA Food Consumption Databases

In [168]:
import requests

In [169]:
class FCD(object):
    FORMAT = "json"
    API_KEY = "ENTER YOUR API KEY HERE!!!"
    API_URL = "http://api.nal.usda.gov/ndb/{}/?format={}&api_key={}"
    def __init__(self):
        super(FCD, self).__init__()

    @staticmethod
    def get_url(command):
        return FCD.API_URL.format(command, FCD.FORMAT, FCD.API_KEY)
        
    @staticmethod
    def find(name):
        """
        searchs for the given food
        
        :return: returns a list of matching food objects 
        """
        base_url = FCD.get_url("search")
        url = base_url + "&q={}".format(name)
        json_response = requests.get(url).json()["list"]["item"]
        return json_response
    
    @staticmethod
    def get_report(ndbno):
        base_url = FCD.get_url("reports")
        url = base_url + "&type=f&ndbno={}".format(ndbno)
        json_response = requests.get(url).json()["report"]
        return json_response

    @staticmethod
    def get_nutrients(ndbno):
        report = FCD.get_report(ndbno)
        return report["food"]["nutrients"]
    
    @staticmethod
    def get_measures(ndbno):
        nutrients = FCD.get_nutrients(ndbno)
        return set(m["label"] for n in nutrients for m in n["measures"])

## Searching food by keywords

In [150]:
for food in FCD.find("apple slice"):
    print("- {} -{}".format(food["ndbno"], food["name"]))

- 45037461 -MICHIGAN APPLES, APPLE SLICES WITH CARAMEL, UPC: 071228174323
- 45037466 -MICHIGAN APPLES, APPLE SLICES WITH GRAPES, UPC: 071228174316
- 45114816 -GRANDMA HOERNER'S, BIG SLICE, KETTLE COOKED APPLES SLICES, CHUNKS & SAUCE, CHERRY VANILLA, UPC: 018522884145
- 45114792 -GRANDMA HOERNER'S, BIG SLICE, KETTLE COOKED APPLES; SLICES, CHUNKS, & SAUCE, CINNAMON FRENCH TOAST, UPC: 018522884176
- 45114818 -GRANDMA HOERNER'S, BIG SLICE, KETTLE COOKED APPLES; SLICES, CHUNKS & SAUCE, ORANGE GINGER, UPC: 018522884152
- 45105319 -CINNAMON APPLE SLICES DRIED FRUIT, UPC: 085239071366
- 45106128 -CINNAMON APPLE SLICES FREEZE DRIED FRUIT, UPC: 085239802045
- 45050451 -APPLE SLICES FREEZE DRIED FRUIT, UPC: 085239174180
- 45105695 -APPLE SLICES FREEZE DRIED FRUIT, UPC: 085239801741
- 45028519 -GREEN ACRES, FUJI APPLE SLICES IN LIGHT SYRUP, UPC: 076963519835
- 45010410 -CRUNCH PAK, APPLE SLICES, UPC: 732313142206
- 45064671 -CRUNCH PAK, ORGANIC SWEET APPLE SLICES, UPC: 732313142435
- 45064682 -CRU

## Fetching measures for a particular food

In [152]:
selected_ndbno = "01009"
FCD.get_measures(selected_ndbno)

{'cubic inch',
 'cup, diced',
 'cup, melted',
 'cup, shredded',
 'oz',
 'slice (1 oz)'}

In [157]:
selected_measure = "cup, diced"
quantity = 2

## Fetching nutrients and calculating intake

In [131]:
def calculate_consumption(ndbno, measure, quantity):
    nutrients = APIWrapper.get_nutrients(ndbno)
    intake = []
    for nutrient in nutrients:
        for i_measure in nutrient["measures"] :
            if i_measure["label"] == measure and i_measure["value"] != 0 :
                intake.append({
                        "label": nutrient["name"], 
                        "unit": nutrient["unit"], 
                        "intake": i_measure["value"] * quantity
                    })
    return intake



In [158]:
calculate_consumption(selected_ndbno, selected_measure, quantity)

[{'intake': 97.74, 'label': 'Water', 'unit': 'g'},
 {'intake': 1066.0, 'label': 'Energy', 'unit': 'kcal'},
 {'intake': 4458.0, 'label': 'Energy', 'unit': 'kJ'},
 {'intake': 60.38, 'label': 'Protein', 'unit': 'g'},
 {'intake': 87.94, 'label': 'Total lipid (fat)', 'unit': 'g'},
 {'intake': 9.8, 'label': 'Ash', 'unit': 'g'},
 {'intake': 8.16, 'label': 'Carbohydrate, by difference', 'unit': 'g'},
 {'intake': 1.26, 'label': 'Sugars, total', 'unit': 'g'},
 {'intake': 0.68, 'label': 'Glucose (dextrose)', 'unit': 'g'},
 {'intake': 0.32, 'label': 'Lactose', 'unit': 'g'},
 {'intake': 0.26, 'label': 'Galactose', 'unit': 'g'},
 {'intake': 1874.0, 'label': 'Calcium, Ca', 'unit': 'mg'},
 {'intake': 0.36, 'label': 'Iron, Fe', 'unit': 'mg'},
 {'intake': 72.0, 'label': 'Magnesium, Mg', 'unit': 'mg'},
 {'intake': 1202.0, 'label': 'Phosphorus, P', 'unit': 'mg'},
 {'intake': 200.0, 'label': 'Potassium, K', 'unit': 'mg'},
 {'intake': 1724.0, 'label': 'Sodium, Na', 'unit': 'mg'},
 {'intake': 9.6, 'label': '

In [167]:
FCD.find("MICHIGAN APPLES, APPLE SLICES WITH CARAMEL,  ")

[{'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'MICHIGAN APPLES, APPLE SLICES WITH CARAMEL, UPC: 071228174323',
  'ndbno': '45037461',
  'offset': 0},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'MICHIGAN APPLES, APPLE SLICES WITH GRAPES, UPC: 071228174316',
  'ndbno': '45037466',
  'offset': 1},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': "DIPPIN' STIX, SLICED APPLES & CARAMEL WITH PEANUTS, UPC: 649632000813",
  'ndbno': '45048208',
  'offset': 2},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': "REICHEL, DIPPIN' STICKS, SLICED APPLES & CARAMEL, UPC: 649632000806",
  'ndbno': '45091341',
  'offset': 3}]

In [170]:
FCD.get_nutrients("45005389")

[{'group': 'Proximates',
  'measures': [{'eqv': 30.0, 'label': 'cup', 'qty': 0.33, 'value': '150'}],
  'name': 'Energy',
  'nutrient_id': '208',
  'unit': 'kcal',
  'value': '500'},
 {'group': 'Proximates',
  'measures': [{'eqv': 30.0, 'label': 'cup', 'qty': 0.33, 'value': '5.00'}],
  'name': 'Protein',
  'nutrient_id': '203',
  'unit': 'g',
  'value': '16.67'},
 {'group': 'Proximates',
  'measures': [{'eqv': 30.0, 'label': 'cup', 'qty': 0.33, 'value': '10.00'}],
  'name': 'Total lipid (fat)',
  'nutrient_id': '204',
  'unit': 'g',
  'value': '33.33'},
 {'group': 'Proximates',
  'measures': [{'eqv': 30.0, 'label': 'cup', 'qty': 0.33, 'value': '12.00'}],
  'name': 'Carbohydrate, by difference',
  'nutrient_id': '205',
  'unit': 'g',
  'value': '40.00'},
 {'group': 'Proximates',
  'measures': [{'eqv': 30.0, 'label': 'cup', 'qty': 0.33, 'value': '2.0'}],
  'name': 'Fiber, total dietary',
  'nutrient_id': '291',
  'unit': 'g',
  'value': '6.7'},
 {'group': 'Proximates',
  'measures': [{'eq

In [171]:
calculate_consumption("45005389","cup",1)

[{'intake': '150', 'label': 'Energy', 'unit': 'kcal'},
 {'intake': '5.00', 'label': 'Protein', 'unit': 'g'},
 {'intake': '10.00', 'label': 'Total lipid (fat)', 'unit': 'g'},
 {'intake': '12.00', 'label': 'Carbohydrate, by difference', 'unit': 'g'},
 {'intake': '2.0', 'label': 'Fiber, total dietary', 'unit': 'g'},
 {'intake': '1.00', 'label': 'Sugars, total', 'unit': 'g'},
 {'intake': '20', 'label': 'Calcium, Ca', 'unit': 'mg'},
 {'intake': '1.08', 'label': 'Iron, Fe', 'unit': 'mg'},
 {'intake': '440', 'label': 'Sodium, Na', 'unit': 'mg'},
 {'intake': '0.0', 'label': 'Vitamin C, total ascorbic acid', 'unit': 'mg'},
 {'intake': '0', 'label': 'Vitamin A, IU', 'unit': 'IU'},
 {'intake': '1.50', 'label': 'Fatty acids, total saturated', 'unit': 'g'},
 {'intake': '0.00', 'label': 'Fatty acids, total trans', 'unit': 'g'},
 {'intake': '0', 'label': 'Cholesterol', 'unit': 'mg'}]